In [38]:
from numpy import *
from math import sqrt

# Input: expects Nx3 matrix of points
# Returns R,t
# R = 3x3 rotation matrix
# t = 3x1 column vector

def rigid_transform_3D(A, B):
    assert len(A) == len(B)

    N = A.shape[0]; # total points

    centroid_A = mean(A, axis=0)
    centroid_B = mean(B, axis=0)
    
    # centre the points
    AA = A - tile(centroid_A, (N, 1))
    BB = B - tile(centroid_B, (N, 1))

    # dot is matrix multiplication for array
    H = transpose(AA) * BB

    U, S, Vt = linalg.svd(H)

    R = Vt.T * U.T
    # special reflection case
    if linalg.det(R) < 0:
       Vt[2,:] *= -1
       R = Vt.T * U.T
    t = -R*centroid_A.T + centroid_B.T

    return R, t

# Test with random data

# Random rotation and translation
R = mat(random.rand(3,3))
t = mat(random.rand(3,1))

# make R a proper rotation matrix, force orthonormal
U, S, Vt = linalg.svd(R)
R = U*Vt

# remove reflection
if linalg.det(R) < 0:
   Vt[2,:] *= -1
   R = U*Vt

# number of points
n = 10

A = mat(random.rand(n,3));
B = R*A.T + tile(t, (1, n))
B = B.T;

# Finished generating random data

# recover the transformation
ret_R, ret_t = rigid_transform_3D(A, B)

A2 = (ret_R*A.T) + tile(ret_t, (1, n))
A2 = A2.T

# Find the error
err = A2 - B

err = multiply(err, err)
err = sum(err)
rmse = sqrt(err/n);

print tile(ret_t, (1, n))

print "Points A"
print A
print ""

print "Points B"
print B
print ""

print "Rotation"
print R
print ""

print "Translation"
print t
print ""

print "RMSE:", rmse
print "If RMSE is near zero, the function is correct!"

[[ 0.58055705  0.58055705  0.58055705  0.58055705  0.58055705  0.58055705
   0.58055705  0.58055705  0.58055705  0.58055705]
 [ 0.56302074  0.56302074  0.56302074  0.56302074  0.56302074  0.56302074
   0.56302074  0.56302074  0.56302074  0.56302074]
 [ 0.95119859  0.95119859  0.95119859  0.95119859  0.95119859  0.95119859
   0.95119859  0.95119859  0.95119859  0.95119859]]
Points A
[[ 0.67831992  0.2947016   0.30324323]
 [ 0.51767215  0.4661054   0.93989767]
 [ 0.33672341  0.26599397  0.07577242]
 [ 0.37946693  0.33454703  0.08305857]
 [ 0.45422435  0.04487615  0.62352145]
 [ 0.27127865  0.98050999  0.7532873 ]
 [ 0.5241908   0.64609404  0.93070554]
 [ 0.01198875  0.01690873  0.1468276 ]
 [ 0.3042946   0.17732595  0.26306415]
 [ 0.16464119  0.33238403  0.40279721]]

Points B
[[ 0.7479083   1.26041953  1.30411769]
 [ 1.37947379  1.40427462  1.10176507]
 [ 0.67438242  0.85832027  1.25758379]
 [ 0.70408029  0.89030563  1.32595478]
 [ 0.92482482  1.25065731  0.87538122]
 [ 1.6101389   1.03

In [40]:
# Finished generating random data

# recover the transformation
ret_R, ret_t = rigid_transform_3D(A, B)
r = zeros((4,4))
r[3,3] = 1
r[:3,:3] = ret_R

t = zeros((4,4))
fill_diagonal(t, 1)
t[:3,3:] = ret_t

tm = t.dot(r)
print(tm)

AE = pad(
            A,
            pad_width= ((0,0),) * (1) + ((0,1),), 
            mode="constant", 
            constant_values=1
        )
AE.dot(tm)

[[-0.33544984  0.56200925  0.7560549   0.58055705]
 [ 0.86869044 -0.12597679  0.47906864  0.56302074]
 [ 0.36448638  0.81748116 -0.44595317  0.95119859]
 [ 0.          0.          0.          1.        ]]


array([[ 0.13899018,  0.59199213,  0.51879711,  1.84817105],
       [ 0.57382816,  1.00056671,  0.1955347 ,  2.45699456],
       [ 0.14573062,  0.21767513,  0.3482198 ,  1.41732189],
       [ 0.1935994 ,  0.23901758,  0.41012859,  1.48766431],
       [ 0.11387907,  0.75934197,  0.08685593,  1.88206207],
       [ 1.03532223,  0.64473778,  0.33890228,  2.42606601],
       [ 0.72464549,  0.97404147,  0.29078932,  2.5533728 ],
       [ 0.06418348,  0.12463648, -0.04831364,  1.15614232],
       [ 0.14784909,  0.36372741,  0.19770044,  1.52672481],
       [ 0.38032407,  0.37993633,  0.10408385,  1.66586285]])

In [49]:
def calc_rmse(m1, m2):
    err = m1 - m2
    err = multiply(err, err)
    err = sum(err)
    return sqrt(err/n)

In [50]:
(ret_R*A.T)# + tile(ret_t, (1, n))

matrix([[ 0.16735125,  0.79891674,  0.09382537,  0.12352324,  0.34426777,
          1.02958185,  0.89093559,  0.11649096,  0.19647419,  0.43611084],
        [ 0.69739879,  0.84125388,  0.29529953,  0.32728489,  0.68763657,
          0.47301199,  0.81983852,  0.0786249 ,  0.36802464,  0.29411707],
        [ 0.3529191 ,  0.15056648,  0.3063852 ,  0.37475619, -0.07581737,
          0.56449496,  0.30417902, -0.04728593,  0.13855757,  0.15209846]])

In [56]:
A22 = AE.dot(tm.T)[:,:3]

In [57]:
calc_rmse(A22, B)

3.5457679953657254e-16

In [62]:
pad(array([[1,2],[3,4]]), pad_width = ((0,1),(0,1)), mode="constant", constant_values=0)

array([[1, 2, 0],
       [3, 4, 0],
       [0, 0, 0]])

In [95]:
-


In [99]:
# Input: expects Nx3 matrix of points
# Returns R,t
# R = 3x3 rotation matrix
# t = 3x1 column vector

def rigid_transform_3D(A, B):
    assert len(A) == len(B)

    N = A.shape[0]; # total points

    centroid_A = mean(A, axis=0)
    centroid_B = mean(B, axis=0)
    
    # centre the points
    AA = A - tile(centroid_A, (N, 1))
    BB = B - tile(centroid_B, (N, 1))

    # dot is matrix multiplication for array
    H = transpose(AA) * BB

    U, S, Vt = linalg.svd(H)

    R = Vt.T * U.T
    # special reflection case
    if linalg.det(R) < 0:
       Vt[2,:] *= -1
       R = Vt.T * U.T
    t = -R*centroid_A.T + centroid_B.T

    return R, t

# Test with random data

# Random rotation and translation
R = mat(random.rand(3,3))
t = mat(random.rand(3,1))

# make R a proper rotation matrix, force orthonormal
U, S, Vt = linalg.svd(R)
R = U*Vt

# remove reflection
if linalg.det(R) < 0:
   Vt[2,:] *= -1
   R = U*Vt

# number of points
n = 10

A = mat(random.rand(n,3));
B = R*A.T + tile(t, (1, n))
B = B.T;

# Finished generating random data

# recover the transformation
solve_3d_transformation(A, B)

# recover the transformation
ret_R, ret_t = rigid_transform_3D(A, B)

A2 = (ret_R*A.T) + tile(ret_t, (1, n))
A2 = A2.T

# Find the error
err = A2 - B

err = multiply(err, err)
err = sum(err)
rmse = sqrt(err/n);

print("OLD METHOD\n")

print "Points A"
print A
print ""

print "Points B"
print B
print ""

print "Rotation"
print R
print ""

print "Translation"
print t
print ""

print "RMSE:", rmse
print "If RMSE is near zero, the function is correct!"

r = zeros((4,4))
r[3,3] = 1
r[:3,:3] = ret_R

t = zeros((4,4))
fill_diagonal(t, 1)
t[:3,3:] = ret_t

tm = t.dot(r)
print(tm)

AE = pad(
            A,
            pad_width= ((0,0),) * (1) + ((0,1),), 
            mode="constant", 
            constant_values=1
        )
A22 = AE.dot(tm.T)[:,:3]


[[ 0.33995945]
 [ 0.42080479]
 [ 0.40620604]]
[[-0.4764414   0.5672664   0.67172347  0.        ]
 [ 0.64185506 -0.29771422  0.70667413  0.        ]
 [ 0.60085412  0.76783792 -0.22225943  0.        ]
 [ 0.          0.          0.          1.        ]]
[[-0.4764414   0.5672664   0.67172347  0.33995945]
 [ 0.64185506 -0.29771422  0.70667413  0.42080479]
 [ 0.60085412  0.76783792 -0.22225943  0.40620604]
 [ 0.          0.          0.          1.        ]]
OLD METHOD

Points A
[[ 0.1796931   0.82220594  0.89739335]
 [ 0.68752836  0.99856767  0.49538035]
 [ 0.5590555   0.2594695   0.40425167]
 [ 0.14142356  0.89748377  0.27778341]
 [ 0.58263144  0.0063525   0.22215305]
 [ 0.1718634   0.37832264  0.32690388]
 [ 0.43607718  0.07733141  0.90069208]
 [ 0.71354312  0.95568195  0.44607042]
 [ 0.61037408  0.28385524  0.12235741]
 [ 0.11684451  0.9988394   0.78629493]]

Points B
[[ 1.32355619  0.92552397  0.94604214]
 [ 0.91160497  0.91488303  1.47594546]
 [ 0.49233593  0.98806382  0.85149862]
 [ 0.

In [96]:
A.shape

(10, 3)

In [100]:
pa = mat([[ 0.26832805,  0.57129996,  0.05275835],
 [ 0.14437605,  0.79728663,  0.0503873 ],
 [ 0.03560124,  0.81480421,  0.27551898],
 [ 0.26079327 , 0.53683492 , 0.30887791],
 [ 0.68142564 , 0.1024622 ,  0.32793634],
 [ 0.53706422 , 0.63448485 , 0.37119874],
 [ 0.91510412 , 0.6630108 ,  0.93860456],
 [ 0.49826643 , 0.43610394 , 0.68186992],
 [ 0.33901613 , 0.76119177 , 0.11922139],
 [ 0.28837246 , 0.92693572 , 0.01138285]])

pb = mat([[ 0.57232678 , 0.01149324 , 1.13154136],
 [ 0.61665812, -0.21100935,  1.25388114],
 [ 0.86457814, -0.21766888 , 1.29013596],
 [ 0.80088173, 0.11991548 , 1.18491826],
 [ 0.63891594 , 0.68518925 , 1.04276649],
 [ 0.71350129 , 0.28620612 , 1.41827874],
 [ 1.0197748 ,  0.75158067 , 1.81236037],
 [ 1.01520482 , 0.48747756 , 1.34184818],
 [ 0.58342516, -0.0235825  , 1.34158791],
 [ 0.50396425 ,-0.1964217 ,  1.41559847]])

solve_3d_transformation(pa, pb)

[[ 0.69418676]
 [ 0.13055762]
 [ 0.52150787]]
[[-0.4899781  -0.06345943  0.86942185  0.        ]
 [ 0.72532967 -0.58290247  0.36622613  0.        ]
 [ 0.48354764  0.81006025  0.33163845  0.        ]
 [ 0.          0.          0.          1.        ]]
[[-0.4899781  -0.06345943  0.86942185  0.69418676]
 [ 0.72532967 -0.58290247  0.36622613  0.13055762]
 [ 0.48354764  0.81006025  0.33163845  0.52150787]
 [ 0.          0.          0.          1.        ]]


(array([[-0.4899781 , -0.06345943,  0.86942185,  0.69418676],
        [ 0.72532967, -0.58290247,  0.36622613,  0.13055762],
        [ 0.48354764,  0.81006025,  0.33163845,  0.52150787],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 3.4077959472498144e-09,
 array([[ 0.57232678,  0.01149324,  1.13154135],
        [ 0.61665812, -0.21100935,  1.25388114],
        [ 0.86457814, -0.21766888,  1.29013596],
        [ 0.80088173,  0.11991548,  1.18491826],
        [ 0.63891594,  0.68518924,  1.04276649],
        [ 0.7135013 ,  0.28620612,  1.41827874],
        [ 1.0197748 ,  0.75158067,  1.81236037],
        [ 1.01520482,  0.48747756,  1.34184818],
        [ 0.58342516, -0.0235825 ,  1.34158791],
        [ 0.50396425, -0.1964217 ,  1.41559847]]))

In [103]:
linspace(1,0,1)

array([ 1.])

In [108]:
g[0,:]

matrix([[ 0.26832805,  0.57129996,  0.05275835]])